<a href="https://colab.research.google.com/github/naimurborno/Projects/blob/main/resnet%20model%20implementation%20without%20identity%20resnet%20block%20with%20convolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tensorflow import keras

In [57]:
class Identity_block(tf.keras.Model):
  def __init__(self,filters,kernel_size):
    super(Identity_block,self).__init__(name='')
    self.conv1=tf.keras.layers.Conv2D(filters,kernel_size,padding='same')
    self.bn1=tf.keras.layers.BatchNormalization()
    self.conv2=tf.keras.layers.Conv2D(filters,kernel_size,padding='same')
    self.bn2=tf.keras.layers.BatchNormalization()
    self.act=tf.keras.layers.Activation('relu')
    self.add=tf.keras.layers.Add()
  def call(self,input_tensor):
    x=self.conv1(input_tensor)
    x=self.bn1(x)
    x=self.act(x)
    x=self.conv2(x)
    x=self.bn2(x)
    x=self.add([x,input_tensor])
    x=self.act(x)
    return x


In [58]:
class Resnet(tf.keras.Model):
  def __init__(self,num_classes):
    super(Resnet,self).__init__()
    self.conv=tf.keras.layers.Conv2D(64,7,padding='same')
    self.norm=tf.keras.layers.BatchNormalization()
    self.act=tf.keras.layers.Activation('relu')
    self.pool=tf.keras.layers.MaxPool2D((3,3))
    self.id1=Identity_block(64,3)
    self.id2=Identity_block(64,3)
    self.avg=tf.keras.layers.GlobalAveragePooling2D()
    self.classifier=tf.keras.layers.Dense(num_classes,activation='softmax')
  def call(self,inputs):
    x=self.conv(inputs)
    x=self.norm(x)
    x=self.act(x)
    x=self.pool(x)
    x=self.id1(x)
    x=self.id2(x)
    x=self.avg(x)
    x=self.classifier(x)
    return x


In [59]:
model=Resnet(10)

In [60]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [33]:
import tensorflow_datasets as tfds
def preprocess(features):
    return tf.cast(features['image'], tf.float32) / 255., features['label']
dataset = tfds.load('mnist', split=tfds.Split.TRAIN, data_dir='./data')
dataset = dataset.map(preprocess).batch(32)


In [62]:
model.fit(dataset,epochs=2)

Epoch 1/2
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0670 - accuracy: 0.9805
Epoch 2/2
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0411 - accuracy: 0.9880
